# Portfolio Forecasting System - Data Exploration

## Task 1: Preprocess and Explore the Data

This notebook implements **Task 1** from the agent.md requirements:
- Fetch historical data for TSLA, BND, and SPY (July 1, 2015 to July 31, 2025)
- Perform comprehensive data cleaning and preprocessing
- Conduct exploratory data analysis (EDA)
- Test for stationarity using ADF tests
- Calculate risk metrics and volatility analysis

**Assets:**
- **TSLA**: High-growth, high-volatility stock (Tesla)
- **BND**: Bond ETF for stability and low risk (Vanguard Total Bond Market)
- **SPY**: S&P 500 ETF for diversified market exposure

## 1. Setup and Imports

In [8]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import sys
import os
from statsmodels.tsa.stattools import adfuller

# Add src to path
sys.path.append('../src')

# Our custom modules
from data.yfinance_client import YFinanceClient
from data.preprocessor import DataPreprocessor
from data.data_validator import DataValidator
from analysis.feature_engineer import FeatureEngineer
from analysis.eda_engine import EDAEngine

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All imports successful!")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ All imports successful!
📅 Analysis Date: 2025-08-10 22:55:29


## 2. Initialize Components

In [2]:
# Initialize our analysis components
print("🔧 Initializing components...")

# Data fetching with caching enabled
client = YFinanceClient(use_cache=True, cache_expiry_hours=24)

# Data processing
validator = DataValidator()
preprocessor = DataPreprocessor(
    handle_missing="interpolate",
    outlier_threshold=3.0,
    scaling_method="standard"
)

# Analysis engines
feature_engineer = FeatureEngineer()
eda_engine = EDAEngine(figsize=(12, 8))

print("✅ All components initialized!")

INFO:data.cache_manager:Cache manager initialized with directory: data\cache
INFO:data.preprocessor:DataPreprocessor initialized
INFO:analysis.feature_engineer:FeatureEngineer initialized
INFO:analysis.eda_engine:EDAEngine initialized


🔧 Initializing components...
✅ All components initialized!


## 3. Data Fetching

In [3]:
# Define our parameters
SYMBOLS = ['TSLA', 'BND', 'SPY']
START_DATE = '2015-07-01'
END_DATE = '2025-07-31'

print(f"📊 Fetching data for: {', '.join(SYMBOLS)}")
print(f"📅 Date range: {START_DATE} to {END_DATE}")

try:
    # Fetch the data
    raw_data = client.fetch_data(
        symbols=SYMBOLS,
        start_date=START_DATE,
        end_date=END_DATE
    )
    print(f"✅ Data fetched successfully!")
    
except Exception as e:
    print(f"⚠️ Using sample data: {e}")
    
    # Create realistic sample data
    dates = pd.date_range(START_DATE, END_DATE, freq='D')
    sample_data = []
    
    # Base prices and volatilities for each asset
    base_prices = {'TSLA': 200, 'BND': 80, 'SPY': 300}
    volatilities = {'TSLA': 0.03, 'BND': 0.005, 'SPY': 0.015}
    
    for symbol in SYMBOLS:
        np.random.seed(42)  # For reproducible results
        n_days = len(dates)
        
        # Generate realistic price series
        returns = np.random.normal(0.0005, volatilities[symbol], n_days)
        prices = [base_prices[symbol]]
        
        for i in range(1, n_days):
            price = prices[-1] * (1 + returns[i])
            prices.append(max(price, 1))
        
        # Create OHLCV data
        df = pd.DataFrame({
            'Date': dates,
            'Open': [p * np.random.uniform(0.995, 1.005) for p in prices],
            'High': [p * np.random.uniform(1.001, 1.02) for p in prices],
            'Low': [p * np.random.uniform(0.98, 0.999) for p in prices],
            'Close': prices,
            'Volume': np.random.randint(1000000, 10000000, n_days),
            'Symbol': symbol
        })
        
        sample_data.append(df)
    
    raw_data = pd.concat(sample_data, ignore_index=True)
    raw_data = raw_data.sort_values(['Date', 'Symbol']).reset_index(drop=True)

# Display basic info
print(f"\n📋 Data Overview:")
print(f"Shape: {raw_data.shape}")
print(f"Date range: {raw_data['Date'].min()} to {raw_data['Date'].max()}")
print(f"Symbols: {raw_data['Symbol'].unique()}")
print(f"Columns: {list(raw_data.columns)}")

raw_data.head(10)

INFO:data.yfinance_client:Fetching data for symbols: ['TSLA', 'BND', 'SPY']
INFO:data.yfinance_client:Date range: 2015-07-01 to 2025-07-31
INFO:data.yfinance_client:Fetching data for TSLA (attempt 1)


📊 Fetching data for: TSLA, BND, SPY
📅 Date range: 2015-07-01 to 2025-07-31


INFO:data.yfinance_client:Successfully fetched 2535 records for TSLA
INFO:data.yfinance_client:Fetching data for BND (attempt 1)
INFO:data.yfinance_client:Successfully fetched 2535 records for BND
INFO:data.yfinance_client:Fetching data for SPY (attempt 1)
INFO:data.yfinance_client:Successfully fetched 2535 records for SPY
INFO:data.yfinance_client:Successfully fetched data for 3 symbols, total records: 7605


Validation Result: FAILED
Errors: 1, Warnings: 0

ERRORS:
  - Missing required columns: {'Adj Close'}
✅ Data fetched successfully!

📋 Data Overview:
Shape: (7605, 10)
Date range: 2015-07-01 00:00:00-04:00 to 2025-07-30 00:00:00-04:00
Symbols: ['BND' 'SPY' 'TSLA']
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Symbol', 'Capital Gains']


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,Capital Gains
0,2015-07-01 00:00:00-04:00,60.794198,60.914475,60.764127,60.816750,5399300,0.163,0.0,BND,0.0
1,2015-07-01 00:00:00-04:00,175.111010,175.363905,174.124732,174.917130,135979900,0.000,0.0,SPY,0.0
2,2015-07-01 00:00:00-04:00,18.073999,18.174667,17.856667,17.943333,31518000,0.000,0.0,TSLA,NaN
3,2015-07-02 00:00:00-04:00,60.937016,61.027228,60.937016,60.967087,1060100,0.000,0.0,BND,0.0
4,2015-07-02 00:00:00-04:00,175.397596,175.566188,174.335441,174.756927,104373700,0.000,0.0,SPY,0.0
5,2015-07-02 00:00:00-04:00,18.680000,18.830000,18.220667,18.667999,107458500,0.000,0.0,TSLA,NaN
6,2015-07-06 00:00:00-04:00,61.222691,61.222691,61.057304,61.177582,2210700,0.000,0.0,BND,0.0
7,2015-07-06 00:00:00-04:00,173.458760,175.043542,173.256441,174.259583,117975400,0.000,0.0,SPY,0.0
8,2015-07-06 00:00:00-04:00,18.591999,18.779333,18.420000,18.648001,61828500,0.000,0.0,TSLA,NaN
9,2015-07-07 00:00:00-04:00,61.410603,61.470739,61.237698,61.237698,4183200,0.000,0.0,BND,0.0


## 4. Data Cleaning and Validation

In [4]:
print("🧹 Data Cleaning and Validation")

# Validate the raw data
validation_result = validator.validate_data(raw_data)
print(f"\n📊 Validation Results:")
validation_result.print_report()

# Clean and preprocess the data
print("\n🔧 Preprocessing data...")
clean_data = preprocessor.preprocess_data(raw_data)

print(f"✅ Data cleaned: {clean_data.shape}")
print(f"Missing values after cleaning: {clean_data.isnull().sum().sum()}")

# Check data types
print("\n📋 Data Types:")
print(clean_data.dtypes)

INFO:data.preprocessor:Starting preprocessing for 7605 rows
INFO:data.preprocessor:Missing values: 2535 -> 2535
INFO:data.preprocessor:Added derived features
INFO:data.preprocessor:Preprocessing completed. Shape: (7605, 12)


🧹 Data Cleaning and Validation

📊 Validation Results:
Validation Result: FAILED
Errors: 1, Warnings: 0

ERRORS:
  - Missing required columns: {'Adj Close'}

🔧 Preprocessing data...
✅ Data cleaned: (7605, 12)
Missing values after cleaning: 2592

📋 Data Types:
Date             datetime64[ns, America/New_York]
Open                                      float64
High                                      float64
Low                                       float64
Close                                     float64
Volume                                      int64
Dividends                                 float64
Stock Splits                              float64
Symbol                                     object
Capital Gains                             float64
Volume_MA                                 float64
Price_Range                               float64
dtype: object


## 5. Feature Engineering

In [5]:
print("⚙️ Feature Engineering")

# Calculate returns and technical indicators
features_data = feature_engineer.calculate_returns(clean_data)
features_data = feature_engineer.calculate_volatility(features_data)
features_data = feature_engineer.calculate_rolling_statistics(features_data)
features_data = feature_engineer.calculate_technical_indicators(features_data)

print(f"✅ Features calculated: {features_data.shape}")
print(f"New columns: {[col for col in features_data.columns if col not in clean_data.columns]}")

# Display sample of features
print("\n📊 Sample Features:")
features_data[['Date', 'Symbol', 'Close', 'Daily_Return', 'Volatility_30d', 'SMA_20', 'RSI']].head()

⚙️ Feature Engineering


AttributeError: 'FeatureEngineer' object has no attribute 'calculate_returns'

## 6. Exploratory Data Analysis (EDA)

In [6]:
print("📈 Exploratory Data Analysis")

# Plot closing prices
print("\n1. Closing Prices Trends")
eda_engine.plot_price_trends(features_data)
plt.show()

# Plot daily returns
print("\n2. Daily Returns Distribution")
eda_engine.plot_returns_distribution(features_data)
plt.show()

# Plot volatility
print("\n3. Volatility Analysis")
eda_engine.plot_volatility_analysis(features_data)
plt.show()

📈 Exploratory Data Analysis

1. Closing Prices Trends


NameError: name 'features_data' is not defined

## 7. Stationarity Testing

In [7]:
print("🔍 Stationarity Testing (ADF Test)")

def perform_adf_test(series, name):
    """Perform Augmented Dickey-Fuller test"""
    result = adfuller(series.dropna())
    print(f"\n{name}:")
    print(f"  ADF Statistic: {result[0]:.6f}")
    print(f"  p-value: {result[1]:.6f}")
    print(f"  Critical Values:")
    for key, value in result[4].items():
        print(f"    {key}: {value:.3f}")
    
    if result[1] <= 0.05:
        print(f"  ✅ Stationary (reject null hypothesis)")
    else:
        print(f"  ❌ Non-stationary (fail to reject null hypothesis)")
    
    return result[1] <= 0.05

# Test stationarity for each symbol
stationarity_results = {}

for symbol in SYMBOLS:
    symbol_data = features_data[features_data['Symbol'] == symbol].copy()
    
    print(f"\n📊 {symbol} Stationarity Tests:")
    
    # Test closing prices
    prices_stationary = perform_adf_test(symbol_data['Close'], f"{symbol} Closing Prices")
    
    # Test daily returns
    returns_stationary = perform_adf_test(symbol_data['Daily_Return'], f"{symbol} Daily Returns")
    
    stationarity_results[symbol] = {
        'prices_stationary': prices_stationary,
        'returns_stationary': returns_stationary
    }

print("\n📋 Stationarity Summary:")
for symbol, results in stationarity_results.items():
    print(f"{symbol}: Prices={'✅' if results['prices_stationary'] else '❌'}, Returns={'✅' if results['returns_stationary'] else '❌'}")

🔍 Stationarity Testing (ADF Test)


NameError: name 'features_data' is not defined

## 8. Risk Metrics Calculation

In [ ]:
print("📊 Risk Metrics Calculation")

# Calculate risk metrics for each symbol
risk_metrics = {}

for symbol in SYMBOLS:
    symbol_data = features_data[features_data['Symbol'] == symbol].copy()
    returns = symbol_data['Daily_Return'].dropna()
    
    # Calculate metrics
    metrics = feature_engineer.calculate_risk_metrics(symbol_data)
    
    # Additional calculations
    annual_return = returns.mean() * 252
    annual_volatility = returns.std() * np.sqrt(252)
    sharpe_ratio = (annual_return - 0.02) / annual_volatility if annual_volatility > 0 else 0
    var_95 = np.percentile(returns, 5)
    max_drawdown = ((symbol_data['Close'] / symbol_data['Close'].expanding().max()) - 1).min()
    
    risk_metrics[symbol] = {
        'Annual Return': f"{annual_return:.2%}",
        'Annual Volatility': f"{annual_volatility:.2%}",
        'Sharpe Ratio': f"{sharpe_ratio:.3f}",
        'VaR (95%)': f"{var_95:.2%}",
        'Max Drawdown': f"{max_drawdown:.2%}"
    }

# Display risk metrics table
risk_df = pd.DataFrame(risk_metrics).T
print("\n📋 Risk Metrics Summary:")
print(risk_df)

# Plot risk-return scatter
plt.figure(figsize=(10, 6))
for symbol in SYMBOLS:
    symbol_data = features_data[features_data['Symbol'] == symbol]
    returns = symbol_data['Daily_Return'].dropna()
    annual_return = returns.mean() * 252
    annual_volatility = returns.std() * np.sqrt(252)
    
    plt.scatter(annual_volatility, annual_return, s=100, label=symbol, alpha=0.7)
    plt.annotate(symbol, (annual_volatility, annual_return), 
                xytext=(5, 5), textcoords='offset points')

plt.xlabel('Annual Volatility')
plt.ylabel('Annual Return')
plt.title('Risk-Return Profile')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 9. Summary and Insights

In [ ]:
print("📝 Task 1 Summary and Key Insights")
print("=" * 50)

print("\n🎯 Data Collection:")
print(f"  • Successfully processed {len(features_data)} records")
print(f"  • Date range: {START_DATE} to {END_DATE}")
print(f"  • Assets analyzed: {', '.join(SYMBOLS)}")

print("\n🧹 Data Quality:")
print(f"  • Data validation: {'✅ Passed' if validation_result.is_valid else '❌ Issues found'}")
print(f"  • Missing values: {clean_data.isnull().sum().sum()} (handled)")
print(f"  • Features engineered: {len([col for col in features_data.columns if col not in raw_data.columns])}")

print("\n📊 Stationarity Analysis:")
for symbol, results in stationarity_results.items():
    price_status = "Stationary" if results['prices_stationary'] else "Non-stationary (needs differencing)"
    return_status = "Stationary" if results['returns_stationary'] else "Non-stationary"
    print(f"  • {symbol}: Prices - {price_status}, Returns - {return_status}")

print("\n💰 Risk Profile Insights:")
for symbol in SYMBOLS:
    metrics = risk_metrics[symbol]
    print(f"  • {symbol}: Return {metrics['Annual Return']}, Volatility {metrics['Annual Volatility']}, Sharpe {metrics['Sharpe Ratio']}")

print("\n🎯 Key Findings for ARIMA Modeling:")
non_stationary_prices = [symbol for symbol, results in stationarity_results.items() if not results['prices_stationary']]
if non_stationary_prices:
    print(f"  • Price series requiring differencing: {', '.join(non_stationary_prices)}")
else:
    print("  • All price series are stationary")

print(f"  • Daily returns are generally stationary for modeling")
print(f"  • TSLA shows highest volatility - suitable for forecasting")
print(f"  • BND provides stability as expected for bond ETF")
print(f"  • SPY offers balanced risk-return profile")

print("\n✅ Task 1 Complete - Ready for Time Series Modeling (Task 2)")